In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
dataset_path = '/content/drive/MyDrive/DS_PROJECT/DS_PROJECT/SolarGuard/dataset/dataset'

In [3]:
import zipfile
import os

zip_paths = [
    f'{dataset_path}/Bird-drop-20250428T115508Z-1-001.zip',
    f'{dataset_path}/Clean-20250428T115502Z-1-001.zip',
    f'{dataset_path}/Dusty-20250428T115457Z-1-001.zip',
    f'{dataset_path}/Electrical-damage-20250428T115452Z-1-001.zip',
    f'{dataset_path}/Physical-Damage-20250428T115448Z-1-001.zip',
    f'{dataset_path}/Snow-Covered-20250428T115440Z-1-001.zip',
]

for zip_file in zip_paths:
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(dataset_path)
        image_files = [f for f in zip_ref.namelist() if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        print(f"{zip_file.split('/')[-1]} → {len(image_files)} image(s)")

Bird-drop-20250428T115508Z-1-001.zip → 207 image(s)
Clean-20250428T115502Z-1-001.zip → 193 image(s)
Dusty-20250428T115457Z-1-001.zip → 190 image(s)
Electrical-damage-20250428T115452Z-1-001.zip → 103 image(s)
Physical-Damage-20250428T115448Z-1-001.zip → 69 image(s)
Snow-Covered-20250428T115440Z-1-001.zip → 123 image(s)


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_height, img_width = 224, 224
batch_size = 32

datagen = train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

num_classes = train_generator.num_classes

Found 711 images belonging to 6 classes.
Found 174 images belonging to 6 classes.


In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras import layers,models

In [7]:
model=models.Sequential()

model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(img_height,img_width,3)))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Flatten())
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dense(train_generator.num_classes,activation='softmax'))

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 186624)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    47,776,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │         1,542 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 47,796,934 (182.33 MB)

 Trainable params: 47,796,934 (182.33 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [9]:
epochs=20
history =model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 94s 4s/step - accuracy: 0.2071 - loss: 7.2754 - val_accuracy: 0.2816 - val_loss: 1.6539
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 90s 4s/step - accuracy: 0.4215 - loss: 1.5360 - val_accuracy: 0.3851 - val_loss: 1.4505
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 120s 5s/step - accuracy: 0.6759 - loss: 0.9979 - val_accuracy: 0.6034 - val_loss: 1.0967
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 129s 5s/step - accuracy: 0.8970 - loss: 0.4308 - val_accuracy: 0.6667 - val_loss: 1.0360
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 112s 5s/step - accuracy: 0.9607 - loss: 0.2178 - val_accuracy: 0.5747 - val_loss: 1.5643
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 105s 4s/step - accuracy: 0.9830 - loss: 0.1033 - val_accuracy: 0.6667 - val_loss: 1.1797
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 103s 5s/step - accuracy: 0.9809 - loss: 0.0822 - val_accuracy: 0.6494 - val_loss: 1.3015
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 105s 5s/step - accuracy: 0.9923 - loss: 0.0468 - val_accuracy: 0.7011 - val

In [10]:
Sequential_loss, Sequential_accuracy = model.evaluate(val_generator)
print(f"Sequential Modl Accuracy: {Sequential_accuracy * 100:.2f}%")

6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.6251 - loss: 1.2935
Sequential Modl Accuracy: 60.34%


In [12]:
model.save('/content/drive/MyDrive/DS_PROJECT/DS_PROJECT/SolarGuard/Sequential_model.h5')

In [14]:
num_classes = 6

resnet_model = Sequential()


pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(224, 224, 3),
    pooling='avg',
    weights='imagenet'
)

# Fixing layer freezing
for layer in pretrained_model.layers:
    layer.trainable = False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(num_classes, activation='softmax'))

resnet_model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 2048)           │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 6)              │         3,078 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,639,878 (93.99 MB)

 Trainable params: 1,052,166 (4.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [15]:
from tensorflow.keras.optimizers import Adam

resnet_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [16]:
epochs=20
history =resnet_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs
)

Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 177s 7s/step - accuracy: 0.2622 - loss: 1.7482 - val_accuracy: 0.2586 - val_loss: 1.5708
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 168s 7s/step - accuracy: 0.3366 - loss: 1.5944 - val_accuracy: 0.3506 - val_loss: 1.4783
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 172s 7s/step - accuracy: 0.4228 - loss: 1.4835 - val_accuracy: 0.5000 - val_loss: 1.3942
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 166s 7s/step - accuracy: 0.4794 - loss: 1.4101 - val_accuracy: 0.4885 - val_loss: 1.3331
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 162s 7s/step - accuracy: 0.4873 - loss: 1.3708 - val_accuracy: 0.5230 - val_loss: 1.3081
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 175s 8s/step - accuracy: 0.5326 - loss: 1.2952 - val_accuracy: 0.5000 - val_loss: 1.3022
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 160s 7s/step - accuracy: 0.5393 - loss: 1.2646 - val_accuracy: 0.4713 - val_loss: 1.3486
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 168s 7s/step - accuracy: 0.4862 - loss: 1.3136 - val_accuracy: 0.4655 - v

In [17]:
resnet_loss, resnet_accuracy = resnet_model.evaluate(val_generator)
print(f"resnet_model Model Accuracy: {resnet_accuracy * 100:.2f}%")

6/6 ━━━━━━━━━━━━━━━━━━━━ 32s 5s/step - accuracy: 0.4828 - loss: 1.3149
resnet_model Model Accuracy: 52.30%


In [18]:
resnet_model.save('/content/drive/MyDrive/DS_PROJECT/DS_PROJECT/SolarGuard/resnet_model.h5')

In [19]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

base_model = MobileNetV2(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # freeze for feature extraction

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

mobilenet_model = Model(inputs=base_model.input, outputs=predictions)

mobilenet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

mobilenet_model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_3[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,422,726 (9.24 MB)

 Trainable params: 164,742 (643.52 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [21]:
mobilenet_history = mobilenet_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20
)

Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 75s 3s/step - accuracy: 0.3648 - loss: 1.8759 - val_accuracy: 0.6437 - val_loss: 0.9070
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - accuracy: 0.6344 - loss: 0.9607 - val_accuracy: 0.7299 - val_loss: 0.7539
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 83s 2s/step - accuracy: 0.7443 - loss: 0.7578 - val_accuracy: 0.7529 - val_loss: 0.6722
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - accuracy: 0.7629 - loss: 0.6633 - val_accuracy: 0.7759 - val_loss: 0.6378
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 84s 2s/step - accuracy: 0.7507 - loss: 0.6634 - val_accuracy: 0.7759 - val_loss: 0.5879
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 58s 3s/step - accuracy: 0.8378 - loss: 0.4979 - val_accuracy: 0.7759 - val_loss: 0.5897
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 61s 3s/step - accuracy: 0.8804 - loss: 0.4106 - val_accuracy: 0.7989 - val_loss: 0.5857
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 69s 2s/step - accuracy: 0.8710 - loss: 0.3904 - val_accuracy: 0.8276 - val_loss:

In [22]:
# Evaluate MobileNetV2 model on validation data
mobilenet_loss, mobilenet_accuracy = mobilenet_model.evaluate(val_generator)
print(f"MobileNetV2 Model Accuracy: {mobilenet_accuracy * 100:.2f}%")

6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.8207 - loss: 0.6485
MobileNetV2 Model Accuracy: 82.76%


In [ ]:
# Classification Report (Precision, Recall, F1-Score)
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=validation.class_indices.keys()))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred_classes)`

# Plot Confusion Matrix
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=validation.class_indices.keys(), yticklabels=validation.class_indices.keys())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [24]:
mobilenet_model.save('/content/drive/MyDrive/DS_PROJECT/DS_PROJECT/SolarGuard/mobile_net.h5')